In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
df = pd.read_csv('../data/processed/train_cleaned.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1460 non-null   int64  
 1   MSSubClass          1460 non-null   int64  
 2   MSZoning            1460 non-null   object 
 3   Street              1460 non-null   object 
 4   LotShape            1460 non-null   object 
 5   LandContour         1460 non-null   object 
 6   Utilities           1460 non-null   object 
 7   LotConfig           1460 non-null   object 
 8   LandSlope           1460 non-null   object 
 9   Neighborhood        1460 non-null   object 
 10  Condition1          1460 non-null   object 
 11  Condition2          1460 non-null   object 
 12  BldgType            1460 non-null   object 
 13  HouseStyle          1460 non-null   object 
 14  OverallQual         1460 non-null   int64  
 15  OverallCond         1460 non-null   int64  
 16  YearBu

# Train-Test split

In [5]:
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

In [6]:
# train-test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [7]:
X_train.shape

(1168, 74)

In [8]:
y_train.shape

(1168,)

In [9]:
X_test.shape

(292, 74)

In [10]:
y_test.shape

(292,)

In [11]:
numerical_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

In [12]:
numerical_features

['Id',
 'MSSubClass',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'MiscVal',
 'MoSold',
 'YrSold',
 'GarageYrBlt_NoNull',
 'MasVnrArea_NoNull',
 'LotFrontage_capped',
 'LotArea_capped',
 'TotalBsmtSF_capped',
 'GrLivArea_capped',
 'GarageArea_capped',
 '1stFlrSF_capped']

In [13]:
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition',
 'GarageType_NoNull']

# ANOVA test

In [14]:
import pandas as pd
from scipy.stats import f_oneway
from sklearn.preprocessing import LabelEncoder

def anova_categorical_features(X_train, y_train, categorical_features):
    """
    Perform ANOVA on categorical features only
    """
    results = {}
    
    for feature in categorical_features:
        # Encode categorical feature to numerical
        le = LabelEncoder()
        encoded_feature = le.fit_transform(X_train[feature].astype(str))
        
        # Group encoded values by target classes
        groups = []
        unique_classes = pd.unique(y_train)
        
        for class_val in unique_classes:
            mask = y_train == class_val
            group_data = encoded_feature[mask]
            groups.append(group_data)
        
        # Perform ANOVA
        f_stat, p_value = f_oneway(*groups)
        results[feature] = {'F-statistic': f_stat, 'p-value': p_value}
        
        print(f"{feature}: F={f_stat:.4f}, p={p_value:.4f}")
    
    return results

# Usage:


In [15]:
results = anova_categorical_features(X_train, y_train, categorical_features)

MSZoning: F=1.7452, p=0.0000
Street: F=1.5896, p=0.0000
LotShape: F=1.3772, p=0.0001
LandContour: F=1.1403, p=0.0564
Utilities: F=0.5223, p=1.0000
LotConfig: F=0.9271, p=0.8194
LandSlope: F=1.3218, p=0.0004
Neighborhood: F=1.4119, p=0.0000
Condition1: F=0.6955, p=1.0000
Condition2: F=0.7471, p=0.9998
BldgType: F=1.0629, p=0.2305
HouseStyle: F=1.1890, p=0.0183
RoofStyle: F=1.1323, p=0.0668
RoofMatl: F=0.9057, p=0.8838
Exterior1st: F=0.9602, p=0.6878
Exterior2nd: F=1.0442, p=0.3005
ExterQual: F=3.2048, p=0.0000
ExterCond: F=0.9551, p=0.7101
Foundation: F=1.3672, p=0.0001
BsmtQual: F=2.1929, p=0.0000
BsmtCond: F=1.0872, p=0.1564
BsmtExposure: F=1.4402, p=0.0000
BsmtFinType1: F=2.1929, p=0.0000
BsmtFinType2: F=0.7921, p=0.9975
Heating: F=1.4108, p=0.0000
HeatingQC: F=1.3030, p=0.0007
CentralAir: F=1.5494, p=0.0000
Electrical: F=1.0447, p=0.2985
KitchenQual: F=2.8033, p=0.0000
Functional: F=0.7851, p=0.9982
FireplaceQu: F=1.6510, p=0.0000
GarageFinish: F=1.6866, p=0.0000
GarageQual: F=1.078

In [16]:
# Filter features with p-value < 0.05
significant_features = [feature for feature, res in results.items() if res['p-value'] < 0.05]

In [17]:
significant_features

['MSZoning',
 'Street',
 'LotShape',
 'LandSlope',
 'Neighborhood',
 'HouseStyle',
 'ExterQual',
 'Foundation',
 'BsmtQual',
 'BsmtExposure',
 'BsmtFinType1',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'KitchenQual',
 'FireplaceQu',
 'GarageFinish',
 'GarageType_NoNull']

In [18]:
X_train.shape

(1168, 74)

In [19]:
# drop non-significant features from X_train and X_test
X_train = X_train[significant_features + numerical_features]
X_test = X_test[significant_features + numerical_features]

In [20]:
X_train.shape

(1168, 54)

In [21]:
X_train.head()

,MSZoning,Street,LotShape,LandSlope,Neighborhood,HouseStyle,ExterQual,Foundation,BsmtQual,BsmtExposure,...,MoSold,YrSold,GarageYrBlt_NoNull,MasVnrArea_NoNull,LotFrontage_capped,LotArea_capped,TotalBsmtSF_capped,GrLivArea_capped,GarageArea_capped,1stFlrSF_capped
254,RL,Pave,Reg,Gtl,NAmes,1Story,TA,CBlock,TA,No,...,6,2010,1957.0,0.0,70.0,8400.0,1314.0,1314.0,294.0,1314.0
1066,RL,Pave,IR1,Gtl,Gilbert,2Story,Gd,PConc,Gd,No,...,5,2009,1993.0,0.0,59.0,7837.0,799.0,1571.0,380.0,799.0
638,RL,Pave,Reg,Gtl,Edwards,1Story,TA,CBlock,Fa,No,...,5,2008,2005.0,0.0,67.0,8777.0,796.0,796.0,0.0,796.0
799,RL,Pave,Reg,Gtl,SWISU,1.5Fin,TA,BrkTil,Gd,No,...,6,2007,1939.0,252.0,60.0,7200.0,731.0,1768.0,240.0,981.0
380,RL,Pave,Reg,Gtl,SWISU,1.5Fin,TA,BrkTil,TA,No,...,5,2010,1924.0,0.0,50.0,5000.0,1026.0,1691.0,308.0,1026.0


# pearsonr, spearmanr

In [22]:
numerical_features

['Id',
 'MSSubClass',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'MiscVal',
 'MoSold',
 'YrSold',
 'GarageYrBlt_NoNull',
 'MasVnrArea_NoNull',
 'LotFrontage_capped',
 'LotArea_capped',
 'TotalBsmtSF_capped',
 'GrLivArea_capped',
 'GarageArea_capped',
 '1stFlrSF_capped']

In [23]:
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [ ]:
from scipy.stats import pearsonr, spearmanr
import pandas as pd

results = []

for col in numerical_features:
    # Drop NaN safely for this column + target
    valid_data = X_train[[col]].join(y).dropna()
    x = valid_data[col]
    target = valid_data[y.name] # Ensure target is a Series

    # Pearson correlation (linear)
    pearson_corr, pearson_p = pearsonr(x, target)

    # Spearman correlation (rank-based, for non-linear monotonic)
    spearman_corr, spearman_p = spearmanr(x, target)

    results.append({
        'Feature': col,
        'Pearson Correlation': pearson_corr,
        'Pearson P-value': pearson_p,
        'Spearman Correlation': spearman_corr,
        'Spearman P-value': spearman_p
    })

# Convert to DataFrame
corr_df = pd.DataFrame(results)

# Add flags for significance
corr_df['Pearson Significant (p<0.05)'] = corr_df['Pearson P-value'] < 0.05
corr_df['Spearman Significant (p<0.05)'] = corr_df['Spearman P-value'] < 0.05

# Sort by absolute correlation strength
corr_df = corr_df.sort_values(by='Pearson Correlation', key=lambda x: abs(x), ascending=False)

print(corr_df)


               Feature  Pearson Correlation  Pearson P-value  \
2          OverallQual             0.785555    2.596140e-245   
33    GrLivArea_capped             0.705882    7.824090e-177   
19          GarageCars             0.640991    3.931616e-136   
32  TotalBsmtSF_capped             0.629145    9.721332e-130   
34   GarageArea_capped             0.628319    2.649247e-129   
35     1stFlrSF_capped             0.606657    2.454636e-118   
13            FullBath             0.552546     2.457699e-94   
17        TotRmsAbvGrd             0.520388     4.890384e-82   
4            YearBuilt             0.516501     1.225755e-80   
5         YearRemodAdd             0.508593     7.567351e-78   
29   MasVnrArea_NoNull             0.457608     1.636095e-61   
18          Fireplaces             0.457549     1.702416e-61   
31      LotArea_capped             0.406206     1.259502e-47   
28  GarageYrBlt_NoNull             0.389268     1.483993e-43   
30  LotFrontage_capped             0.362

In [26]:
# Optional: show only significant features
significant_features = corr_df[corr_df['Pearson P-value'] < 0.05]['Feature'].tolist()
print("\n✅ Significant numerical features correlated with target:", significant_features)


✅ Significant numerical features correlated with target: ['OverallQual', 'GrLivArea_capped', 'GarageCars', 'TotalBsmtSF_capped', 'GarageArea_capped', '1stFlrSF_capped', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea_NoNull', 'Fireplaces', 'LotArea_capped', 'GarageYrBlt_NoNull', 'LotFrontage_capped', 'BsmtFinSF1', 'WoodDeckSF', '2ndFlrSF', 'OpenPorchSF', 'HalfBath', 'BsmtFullBath', 'BsmtUnfSF', 'BedroomAbvGr', 'EnclosedPorch', 'KitchenAbvGr', 'ScreenPorch', 'MSSubClass', 'OverallCond']
